# PyMem3DG Tutorial 2 - Ensure Mesh Quality
`Cuncheng Zhu, Christopher T. Lee`

This tutorial demonstrate the complementary functionalities of PyMem3DG to ensure the quality of mesh. The tutorial does not mean to be extensive but to provide the flavor and methods that PyMem3DG uses. The extensive documentations is hosted at https://rangamanilabucsd.github.io/Mem3DG/.

To demonstrate, we set up the system to solve the remaining problem from Tutorial 1, evolving closed spherical membrane with high curvature deformation. Again, the following integration is pre-runned. Uncomment $\texttt{fe.integrate()}$ to rerun them.

In [2]:
import pymem3dg.util as dg_util
import pymem3dg as dg

icoFace, icoVertex = dg.getIcosphere(1, 3)
icoVertex = dg_util.sphericalHarmonicsPerturbation(icoVertex, 5, 6, 0.1)
p = dg.Parameters()
p.bending.Kbc = 8.22e-5
p.tension.Ksg = 0.1
p.tension.At = 12.4866
p.osmotic.isPreferredVolume = True
p.osmotic.Kv = 0.02
p.osmotic.Vt = 0.7 * 3.14 * 4 / 3


The setup is exactly the same as the last example in tutorial 1. Without any mesh curation, the mesh will be ill-formed.

## Spring regularization
The first strategy is to regularize the mesh, restricting the tangential movement of vertices. Regularization does not change the mesh connectivity and the number of elements in the mesh, which leads to benefit of efficiency and ease of output. The three type of regularization provided by PyMem3DG includes constraints on edge length, local triangle area, and local corner angles, adjusted by $K_{se}$, $K_{sl}$ and $K_{st}$, respectively. 

In [4]:
outputDir="output/tutorial2/ico_reg"
p.spring.Kst = 1e-7
p.spring.Ksl = 1e-5
p.spring.Kse = 1e-7
g = dg.System(icoFace, icoVertex, p)
g.initialize()
fe = dg.Euler(g, 0.1, 50000, 1000, 0, outputDir)
fe.ifPrintToConsole = True
fe.ifOutputTrajFile = True
# fe.integrate()


area_init = 12.6076
vol_init = 4.15656
Characteristic volume wrt to At = 4.14897
Initialized NetCDF file at output/tutorial2/ico_reg/traj.nc

t: 0, n: 0, isSmooth: 1
A, At: 12.6076, 12.4866, V, Vt: 4.15656, 2.93067, h: 1
E_total: 0.00299491
E_kin: 0
E_pot: 0.00299491
W_ext: 0
|e|Mech: 0.00258762
|e|Chem: 0
H: [0.116317,1.93667]
K: [-0.183331,3.11045]
phi: [1,1]
sum_phi: 12.6076

t: 1000.27, n: 1, isSmooth: 1
A, At: 12.328, 12.4866, V, Vt: 4.04413, 2.93067, h: 0.990567
E_total: 0.00264987
E_kin: 3.64699e-09
E_pot: 0.00264987
W_ext: 0
|e|Mech: 8.53436e-05
|e|Chem: 0
H: [0.486008,1.58537]
K: [0.169345,2.48857]
phi: [1,1]
sum_phi: 12.328

t: 2002.48, n: 2, isSmooth: 1
A, At: 12.3274, 12.4866, V, Vt: 4.05125, 2.93067, h: 0.99091
E_total: 0.00264409
E_kin: 2.24516e-09
E_pot: 0.00264409
W_ext: 0
|e|Mech: 6.69493e-05
|e|Chem: 0
H: [0.597828,1.47072]
K: [0.310861,2.15152]
phi: [1,1]
sum_phi: 12.3274

t: 3003.51, n: 3, isSmooth: 1
A, At: 12.327, 12.4866, V, Vt: 4.05572, 2.93067, h: 0.990846
E_to

False

067, h: 0.881194
E_total: 0.00241591
E_kin: 1.452e-09
E_pot: 0.00241591
W_ext: 0
|e|Mech: 5.38673e-05
|e|Chem: 0
H: [-1.09198,2.33587]
K: [-1.34324,5.03883]
phi: [1,1]
sum_phi: 12.3881

t: 46119.5, n: 46, isSmooth: 1
A, At: 12.3897, 12.4866, V, Vt: 3.70382, 2.93067, h: 0.881167
E_total: 0.00241331
E_kin: 1.15994e-09
E_pot: 0.00241331
W_ext: 0
|e|Mech: 4.81441e-05
|e|Chem: 0
H: [-1.12154,2.35888]
K: [-1.3908,5.1376]
phi: [1,1]
sum_phi: 12.3897

t: 47123.4, n: 47, isSmooth: 1
A, At: 12.3911, 12.4866, V, Vt: 3.69365, 2.93067, h: 0.881001
E_total: 0.00241122
E_kin: 9.37534e-10
E_pot: 0.00241122
W_ext: 0
|e|Mech: 4.32816e-05
|e|Chem: 0
H: [-1.14629,2.37951]
K: [-1.45755,5.22839]
phi: [1,1]
sum_phi: 12.3911

t: 48125, n: 48, isSmooth: 1
A, At: 12.3922, 12.4866, V, Vt: 3.68484, 2.93067, h: 0.880709
E_total: 0.00240952
E_kin: 7.71508e-10
E_pot: 0.00240952
W_ext: 0
|e|Mech: 3.92618e-05
|e|Chem: 0
H: [-1.16672,2.39796]
K: [-1.5183,5.31165]
phi: [1,1]
sum_phi: 12.3922

t: 49129.4, n: 49, isSmooth

In [8]:
import pymem3dg.visual as dg_vis
import polyscope as ps
dg_vis.animate(outputDir+"/traj.nc", meanCurvature = True)
ps.show()

With the above regularization, the mesh looks significantly nicer, with all triangles closed to equilateral.

Free free to adjust the values of each type of regularization and attain intuition on the behavior. 

As you might observe, there are several disadvantages using the method of regularization. The behavior of simulation will depend on the strength of regularization. Local angle penalty is less restrictive and will minimally affect the underlying physics, but the resolution on high curvature region becomes very coarse. We could combine it with penalty on local area and more strongly edge length, but its influence on physics and optimization can not be neglected. In summary, how to find good balance between restriction and flexibility is not obvious. 

## Vertex shift
In addition, instead of constantly applying regularization force, one could also regularize the mesh once in a while, by "centering" the vertices (*barycenter* to be exact). This can be done by toggling the vertex shift option as follows,

In [7]:
outputDir = "output/tutorial2/ico_shift"
p.spring.Kst = 1e-7
p.spring.Ksl = 1e-5
p.spring.Kse = 1e-7
g = dg.System(icoFace, icoVertex, p)
g.meshProcessor.meshMutator.isShiftVertex = True
g.initialize()
fe = dg.Euler(g, 0.1, 50000, 1000, 0, outputDir)
fe.ifPrintToConsole = True
fe.ifOutputTrajFile = True
fe.processMeshPeriod = 1000
# fe.integrate()


area_init = 

False

12.6076
vol_init = 4.15656
Characteristic volume wrt to At = 4.14897
Initialized NetCDF file at output/tutorial2/ico_shift/traj.nc

t: 0, n: 0, isSmooth: 1
A, At: 12.6076, 12.4866, V, Vt: 4.15656, 2.93067, h: 1
E_total: 0.00299491
E_kin: 0
E_pot: 0.00299491
W_ext: 0
|e|Mech: 0.00258762
|e|Chem: 0
H: [0.116317,1.93667]
K: [-0.183331,3.11045]
phi: [1,1]
sum_phi: 12.6076

t: 1000.27, n: 1, isSmooth: 1
A, At: 12.328, 12.4866, V, Vt: 4.04413, 2.93067, h: 0.990567
E_total: 0.00264987
E_kin: 3.64699e-09
E_pot: 0.00264987
W_ext: 0
|e|Mech: 8.53436e-05
|e|Chem: 0
H: [0.486008,1.58537]
K: [0.169345,2.48857]
phi: [1,1]
sum_phi: 12.328

t: 2002.48, n: 2, isSmooth: 1
A, At: 12.3274, 12.4866, V, Vt: 4.05125, 2.93067, h: 0.99091
E_total: 0.00264409
E_kin: 2.24516e-09
E_pot: 0.00264409
W_ext: 0
|e|Mech: 6.69493e-05
|e|Chem: 0
H: [0.597828,1.47072]
K: [0.310861,2.15152]
phi: [1,1]
sum_phi: 12.3274

t: 3003.51, n: 3, isSmooth: 1
A, At: 12.327, 12.4866, V, Vt: 4.05572, 2.93067, h: 0.990846
E_total: 0.002

which also lead to a good mesh

## Mesh mutation

To resolve the challenges of regularization, PyMem3DG supports adaptive mesh by mesh mutation, including edge flip, edge split and edge collapse. Because of challenges mentioned above, mesh mutation should always be turned on when running complex simulation with large deformation. 

Notice that mesh mutation will most likely increase the computational cost and size of output files. 

The computational cost involves the time to loop through elements and decide whether conduct mesh mutation, and the subsequent overheads needed for recomputation of cached quantities. Such operation happens in the frequency of data output, therefore increasing the number of data output could better prevent the deterioration of mesh, but will increase computation.

Because of the varying mesh sizes, instead of single trajectory file using high performance *NetCDF* file, output file consists of series of snapshots in $\texttt{.ply}$ format. The detail of output files and visualization will be covered in the other tutorial. 

In [9]:
outputDir = "output/tutorial2/ico_mut"
g = dg.System(icoFace, icoVertex, p)
g.meshProcessor.meshMutator.isShiftVertex = True
g.meshProcessor.meshMutator.flipNonDelaunay = True
g.meshProcessor.meshMutator.splitFat = True
g.meshProcessor.meshMutator.splitSkinnyDelaunay = True
g.meshProcessor.meshMutator.splitCurved = True
g.meshProcessor.meshMutator.minimumEdgeLength = 0.001
g.meshProcessor.meshMutator.curvTol = 0.005
g.meshProcessor.meshMutator.collapseSkinny = True
g.meshProcessor.meshMutator.collapseSmall = True
g.meshProcessor.meshMutator.collapseFlat = True
g.meshProcessor.meshMutator.targetFaceArea = 0.0003
g.meshProcessor.meshMutator.isSmoothenMesh = True
g.initialize()
fe = dg.Euler(g, 1, 50000, 1000, 0, outputDir)
fe.ifPrintToConsole = True
fe.ifOutputTrajFile = True
fe.processMeshPeriod = 100
# fe.integrate()


area_init = 12.6076
vol_init = 4.15656
Characteristic volume wrt to At = 4.14897
Initialized NetCDF file at output/tutorial2/ico_mut/traj.nc

t: 0, n: 0, isSmooth: 1
A, At: 12.6076, 12.4866, V, Vt: 4.15656, 2.93067, h: 1
E_total: 0.00299491
E_kin: 0
E_pot: 0.00299491
W_ext: 0
|e|Mech: 0.00258762
|e|Chem: 0
H: [0.116317,1.93667]
K: [-0.183331,3.11045]
phi: [1,1]
sum_phi: 12.6076

t: 1026.29, n: 1, isSmooth: 1
A, At: 12.3279, 12.4866, V, Vt: 4.04438, 2.93067, h: 0.990583
E_total: 0.00264966
E_kin: 3.64116e-09
E_pot: 0.00264966
W_ext: 0
|e|Mech: 8.47248e-05
|e|Chem: 0
H: [0.490019,1.58174]
K: [0.173937,2.47763]
phi: [1,1]
sum_phi: 12.3279

t: 2060.57, n: 2, isSmooth: 1
A, At: 12.3274, 12.4866, V, Vt: 4.05161, 2.93067, h: 0.990926
E_total: 0.00264381
E_kin: 2.21247e-09
E_pot: 0.0026438
W_ext: 0
|e|Mech: 6.59155e-05
|e|Chem: 0
H: [0.604181,1.46412]
K: [0.319143,2.13274]
phi: [1,1]
sum_phi: 12.3274

t: 3060.78, n: 3, isSmooth: 0
A, At: 12.3244, 12.4866, V, Vt: 4.055, 2.93067, h: 0.990744
E_t

False

2.18322]
K: [-1.24548,4.20518]
phi: [1,1]
sum_phi: 12.3835

t: 38514, n: 38, isSmooth: 0
A, At: 12.3861, 12.4866, V, Vt: 3.72582, 2.93067, h: 0.891575
E_total: 0.0024125
E_kin: 2.46152e-09
E_pot: 0.0024125
W_ext: 0
|e|Mech: 7.00089e-05
|e|Chem: 0
H: [-1.58485,2.22107]
K: [-1.33757,4.34528]
phi: [1,1]
sum_phi: 12.3861

t: 39522.4, n: 39, isSmooth: 0
A, At: 12.3883, 12.4866, V, Vt: 3.70858, 2.93067, h: 0.889861
E_total: 0.00240802
E_kin: 1.98248e-09
E_pot: 0.00240802
W_ext: 0
|e|Mech: 6.27646e-05
|e|Chem: 0
H: [-1.62346,2.25497]
K: [-1.43093,4.47566]
phi: [1,1]
sum_phi: 12.3883

t: 40538.6, n: 40, isSmooth: 0
A, At: 12.3901, 12.4866, V, Vt: 3.69377, 2.93067, h: 0.887814
E_total: 0.00240471
E_kin: 1.36218e-09
E_pot: 0.00240471
W_ext: 0
|e|Mech: 5.20507e-05
|e|Chem: 0
H: [-1.65661,2.28468]
K: [-1.50875,4.59307]
phi: [1,1]
sum_phi: 12.3901

t: 41552, n: 41, isSmooth: 0
A, At: 12.3916, 12.4866, V, Vt: 3.68184, 2.93067, h: 0.885526
E_total: 0.00240201
E_kin: 1.19578e-09
E_pot: 0.00240201
W_ex

$\texttt{meshMutator}$ is used to specify conditions for mesh mutation, which should be specified after the instantiation of $\texttt{System}$. For details, please refer to the documentation. Mesh remains well-conditioned and well-resoluted during the simulation.

## Additional notes: 
 
### Reference mesh when mutation
Similarly, in theory we could also support specifying additional reference mesh. However, it is only necessary if the current input mesh has large deviation in total surface area from the reference mesh. Since membrane is nearly unstretchable, normally total surface area remains and self-referencing is sufficient. At the time of writing the tutorial, PyMem3DG will throw run-time error when the topology of reference mesh does not agree with the input mesh.

### Regularization + mutation
PyMem3DG does not recommend using regularization in conjunction with mesh mutation because it is most likely unnecessary to do so (as we see in previous example, vertex shift somewhat fills the role of regularization) and the behavior is not fully tested.

**(Updated)** Spring can be applied in conjunction with mesh mutation. The reference configuration is updated as an copy of current configuration at every mesh mutation period. 